In [1]:
import subprocess
import pandas as pd
import os
import joblib
import shutil

In [2]:
java_code_labels = "C:/Users/shanu/Documents/BTP/CodeSmell-3/data/dataset/dataclass_labels.csv"
java_code_folder = "C:/Users/shanu/Documents/BTP/CodeSmell-3/data/java_test"
java_code_train = "C:/Users/shanu/Documents/BTP/CodeSmell-3/data/java_train"

ck_jar_path = 'ck-0.7.1.jar'
ck_output_folder = 'ck_output'

preprocessing_pipeline_path = 'output/preprocessing_pipeline.joblib'
selected_features_path = 'output/selected_features.joblib'
final_model_path = 'output/final_model.joblib'

In [3]:
print("Running CK tool to extract metrics...")
os.makedirs(ck_output_folder, exist_ok=True)

ck_command = [
    'java',
    '-jar',
    os.path.join("..", ck_jar_path),
    java_code_folder,
    'false',
    '0',
    'false'
]

try:
    subprocess.run(ck_command, check=True, capture_output=True, text=True, cwd=ck_output_folder)
    print(f"CK tool finished. Output files saved in: {ck_output_folder}")
except subprocess.CalledProcessError as e:
    print(f"Error running CK tool: {e}")
    print(f"Stdout: {e.stdout}")
    print(f"Stderr: {e.stderr}")
    os.path.exists(ck_output_folder) and shutil.rmtree(ck_output_folder)

Running CK tool to extract metrics...
CK tool finished. Output files saved in: ck_output


In [4]:
def remove_prefix_path(full_path, prefix_path):
    return os.path.relpath(full_path, prefix_path)

In [5]:
code_labels_df = pd.read_csv(java_code_labels)
code_labels = {
    (remove_prefix_path(row['file'], java_code_train), row['class']): row['label'] for _, row in code_labels_df.iterrows()
}

In [6]:
print("Running Code Smell Detection...", end="\n\n")

try:
    loaded_pipeline = joblib.load(preprocessing_pipeline_path)
    selected_features = joblib.load(selected_features_path)
    loaded_model = joblib.load(final_model_path)

    new_data = pd.read_csv(os.path.join(ck_output_folder, "class.csv"))

    missing_features = [feature for feature in selected_features if feature not in new_data.columns]
    if missing_features:
        print(f"Error: The following selected features are missing in the new data: {missing_features}")
        print("Please ensure the CK output contains the necessary metrics.")

    numerical_cols = new_data.select_dtypes(include=['number']).columns.tolist()
    new_data[numerical_cols] = loaded_pipeline.transform(new_data[numerical_cols])
    new_data_final = new_data[selected_features]
    predictions = loaded_model.predict(new_data_final)

    correct, total = 0, 0
    
    print(f"Code Smell Predictions for the Java classes ({java_code_folder}):")
    for index, prediction in enumerate(predictions):
        file_name = remove_prefix_path(new_data['file'].iloc[index], java_code_folder)
        class_name = new_data['class'].iloc[index]
        print((file_name, class_name) in code_labels)
        actual = code_labels[(file_name, class_name)]
        if prediction==actual:
            correct += 1
        total += 1
        print(f"File: {file_name}, Class: {class_name}, Predicted Label: {prediction}, Actual Label: {actual}")
    
    print(f"Correct {correct}/{total} = {int(correct/total*100)}%")

except FileNotFoundError as e:
    print(f"Error: Could not find saved model or features: {e.filename}")
except Exception as e:
    print(f"An error occurred during prediction: {e}")
finally:
    os.path.exists(ck_output_folder) and shutil.rmtree(ck_output_folder)

Running Code Smell Detection...

Code Smell Predictions for the Java classes (C:/Users/shanu/Documents/BTP/CodeSmell-3/data/java_test):
True
File: unbounded_knapsack.java, Class: chapter_dynamic_programming.unbounded_knapsack, Predicted Label: 1, Actual Label: 1
True
File: CellTest.java, Class: com.iluwatar.typeobject.CellTest, Predicted Label: 0, Actual Label: 0
True
File: OmsOrderController.java, Class: com.macro.mall.controller.OmsOrderController, Predicted Label: 1, Actual Label: 1
True
File: SmsCouponHistoryController.java, Class: com.macro.mall.controller.SmsCouponHistoryController, Predicted Label: 1, Actual Label: 1
True
File: SmsFlashPromotionMapper.java, Class: com.macro.mall.mapper.SmsFlashPromotionMapper, Predicted Label: 0, Actual Label: 0
True
File: WeatherObserver.java, Class: com.iluwatar.observer.WeatherObserver, Predicted Label: 0, Actual Label: 0
True
File: BullyMessageManagerTest.java, Class: com.iluwatar.leaderelection.bully.BullyMessageManagerTest, Predicted Label